#### NLP | Model

# Coronavirus Tweets: April 2020<a id='top'></a> 

### Natural Language Processing Stepwise Analysis<a id='top'></a> 

1. [Research Question](#1)<br/>
2. [DataFrames](#2) <br/>
3. [Exploratory Data Analysis](#3)<br/>
   [Data Summary](#31)<br/>
   [Text Preprocessing](#32)<br/>
4. [Vectorizer](#4)<br/>
5. [Topic Modeling/Dimensionality Reduction](#5)<br/>
6. [Sentiment Analysis](#6)<br/>
7. [Classification](#7) <br/>
    1 [Naive Bayes: Bernoulli](#71)<br/>
    2 [Naive Bayes: Gaussian](#72)<br/>
    3 [Naive Bayes: Multinomial](#73)<br/>

In [44]:
import glob 
import nltk
import matplotlib.pyplot as plt
import numpy as np
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import pandas as pd
import pickle
import re
import seaborn as sns
import string
pd.set_option('display.max_colwidth', None)
%matplotlib inline
%config InlineBackend.figure_formats = ['retina']  # or svg
sns.set(context='notebook', style='whitegrid')


from cleantext import clean
# from itertools import cycle
from nltk.tokenize import MWETokenizer, word_tokenize
from nltk.tag import pos_tag
from nltk.stem.lancaster import LancasterStemmer
# from nltk.tokenize import word_tokenize, RegexpTokenizer
# from nltk.util import ngrams
# from sklearn import svm
from sklearn.decomposition import PCA, NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import confusion_matrix, accuracy_score
# from sklearn.model_selection import train_test_split
# from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 







# 1 | Research Design<a id='1'></a> 

* **Research Question:** What were Americans tweeting about coronavirus and COVID-19 in April 2020? 
* **Impact Hypothesis:** Inform CDC's communication strategy for future pandemics. 
* **Data source:** Coronavirus COVID-19 Tweets [early](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-early-april) and [late](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-late-april) April, n=???


[back to top](#top)

# 2 | [DataFrames](https://github.com/slp22/nlp-project/blob/main/nlp-coronavirus-tweets-mvp.ipynb)<a id='2'></a> 

In [ ]:
# load clean tweet corpus from mvp 
df = pd.read_csv('./raw_data/tweet_df.csv', low_memory=False)


In [ ]:
df.head(3)

[back to top](#top)

# 3 | Exploratory Data Analysis<a id='3'></a> 

##### Note: Full EDA part of [MVP](https://github.com/slp22/nlp-project/blob/main/nlp-coronavirus-tweets-mvp.ipynb).

### 3.1 Data Summary<a id='31'></a> 

In [ ]:
# dataframe info
df.info()


#### No missing data, all data types are ????.

[back to top](#top)

### 3.2 Text Preprocessing<a id='32'></a>  

In [18]:
# drop all columns except text
df = df.drop(columns=['created_at', 'screen_name', 'country_code', 
                 'account_lang', 'verified', 'lang'])
df.head(2)

,text
0,....#SUNDAYFUNDAY #coronavirus style #vino cheers 🍷 https://t.co/SrymChBkq2
1,"This pandemic has confirmed my worst fears, most people don’t know how to make entertaining videos. #Covid_19 #SinceIveBeenQuarantined #AmericasUnfunniestVideos #WrestleMania #tonyaharding"


In [19]:
# remove numbers, punctuation, and capital letters
alphanumeric = lambda x: re.sub('\w*\d\w*',' ', str(x))
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
                          
df['text'] = df.text.map(alphanumeric).map(punc_lower)
df.head(2)

,text
0,sundayfunday coronavirus style vino cheers 🍷 https t co
1,this pandemic has confirmed my worst fears most people don’t know how to make entertaining videos sinceivebeenquarantined americasunfunniestvideos wrestlemania tonyaharding


In [20]:
# remove emojis
df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
df.head(2)

,text
0,sundayfunday coronavirus style vino cheers https t co
1,this pandemic has confirmed my worst fears most people dont know how to make entertaining videos sinceivebeenquarantined americasunfunniestvideos wrestlemania tonyaharding


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138796 entries, 0 to 138795
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    138796 non-null  object
dtypes: object(1)
memory usage: 1.1+ MB


In [22]:
# # save preprocessed corpus as corpus_tweets_df
# corpus_tweets_df = df 
# corpus_tweets_df.to_pickle('./raw_data/corpus_tweets_df.pkl')
# corpus_tweets_df.to_csv(r'//Users/sandraparedes/Documents/GitHub/metis_dsml/05_nlp/g00-nlp-project/raw_data/corpus_tweets_df.csv', index=False)


[back to top](#top)

# New things to try:

In [1]:
# https://stackoverflow.com/questions/5511708/adding-words-to-nltk-stoplist 

# stopwords = nltk.corpus.stopwords.words('english')
# newStopWords = ['stopWord1','stopWord2']
# stopwords.extend(newStopWords)


In [ ]:
# http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g01-nlp-exercises/02-nlp-pipeline/nlp_pipeline-solutions.ipynb

In [ ]:
# 'the' in vec.get_feature_names()

In [ ]:
# from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer

# stemmer = SnowballStemmer("english")

# def prep(word, stemmer=None):
#     with open('./stop_words_english.txt', 'r') as f:
#         stopwords = [s.strip() for s in f.readlines()]
#     if word.lower() in stopwords:
#         return None
#     elif stemmer is None:
#         return word.lower()
#     else:
#         return stemmer.stem(word)

# vec = CountVectorizer(stop_words='english',
#                       min_df=1,  # This is default; this is just a reminder it exists
#                       max_df=0.8,
#                       preprocessor=prep)


[back to top](#top)

## 4 | Vectorizer<a id='4'></a>  

### Term Frequency Inverse Document Frequency (TF-IDF)

In [30]:
# option tfidf2 = TfidfVectorizer(ngram_range=(1,2), binary=True, stop_words='english')
tf_vectorizer = TfidfVectorizer(stop_words='english', min_df=0.01, max_df=.95)
tf_vectorizer

TfidfVectorizer(max_df=0.95, min_df=0.01, stop_words='english')

In [31]:
# document-term matrix with TF-IDF
tf_doc_term_mtx = tf_vectorizer.fit_transform(corpus)
type(tf_doc_term_mtx)

scipy.sparse._csr.csr_matrix

In [32]:
tf_doc_term_df = pd.DataFrame(tf_doc_term_mtx.toarray(), 
                              columns=tf_vectorizer.get_feature_names_out())
tf_doc_term_df.head(2)

,america,americans,amp,april,away,best,better,business,california,care,...,watch,way,week,weeks,work,workers,working,world,year,york
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


[back to top](#top)

# New things to try:

In [ ]:
# # http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g1-nlp-overview/nlp_unsupervised_nlp_exercises6_soln.ipynb

# word_list = MWETokenizer([('I','am')]).tokenize(word_tokenize(text))
# word_list

In [ ]:
# r_list = [word for word in word_list if word[0] == 'r']
# r_list

In [ ]:
# # Use the `LancasterStemmer` to find the base of these five words. Hint: you'll need to write a `for` loop to stem each item in the list. Save the new stemmed list in a variable called `stemmed_list`.

# stemmer = LancasterStemmer()
# stemmed_list = []
# for word in r_list:
#     stemmed_word = stemmer.stem(word)
#     stemmed_list.append(stemmed_word)

# stemmed_list

In [ ]:
## Find the part of speech of each item in the `r_list` using `pos_tag`. Save the results in a variable called `pos_list`.
# pos_list = pos_tag(r_list)
# pos_list

## 5 | Topic Modeling/Dimensionality Reduction <a id='5'></a>  

### Non-Negative Matrix Factorization (NMF)

In [38]:
# V     visible variables     doc_term             input (corpus matrix)
# W     weights               doc_topic            feature set
# H     hidden variables      topic_term           coefficients

In [39]:
V = tf_doc_term_mtx #
V.shape

(138796, 146)

In [40]:
# W matrix = feature set & weights

nmf = NMF(n_components=5, init=None)
W = nmf.fit_transform(V).round(3)
print(type(W))
W.shape

<class 'numpy.ndarray'>


(138796, 5)

In [41]:
# H matrix = hidden variables & coefficients 

H = pd.DataFrame(nmf.components_.round(2),
                 index = ['c1', 'c2','c3', 'c4', 'c5'],
                 columns = tf_vectorizer.get_feature_names_out())
print('H.shape:',  H.shape)
H.T.style.background_gradient(cmap='Blues')


H.shape: (5, 146)


,c1,c2,c3,c4,c5
america,0.020000,0.050000,0.070000,0.190000,0.020000
americans,0.000000,0.070000,0.120000,0.260000,0.000000
amp,0.000000,0.000000,8.040000,0.000000,0.000000
april,0.060000,0.030000,0.090000,0.040000,0.200000
away,0.000000,0.020000,0.050000,0.190000,0.070000
best,0.040000,0.020000,0.070000,0.160000,0.100000
better,0.030000,0.020000,0.060000,0.230000,0.080000
business,0.060000,0.020000,0.110000,0.150000,0.040000
california,0.100000,0.070000,0.000000,0.020000,0.260000
care,0.050000,0.000000,0.180000,0.370000,0.060000


In [42]:
# function to display topics
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))


#### Top terms by topic:

In [43]:
display_topics(nmf, tf_vectorizer.get_feature_names_out(), 10)


Topic  0
https, thank, pandemic, coronaviruspandemic, great, thanks, stayhomestaysafe, join, check, support

Topic  1
coronavirus, trump, lockdown, death, realdonaldtrump, news, covid, million, china, pandemic

Topic  2
amp, thank, health, today, support, work, workers, help, need, community

Topic  3
people, just, pandemic, like, coronaviruspandemic, realdonaldtrump, trump, time, need, help

Topic  4
new, quarantine, stayhome, covid, day, home, today, stay, cases, york


[back to top](#top)

## 6 | Sentiment Analysis<a id='6'></a>  

In [23]:
#read in corpus
df = pd.read_pickle("./raw_data/corpus_tweets_df.pkl")  
df.head(2)

,text
0,sundayfunday coronavirus style vino cheers https t co
1,this pandemic has confirmed my worst fears most people dont know how to make entertaining videos sinceivebeenquarantined americasunfunniestvideos wrestlemania tonyaharding


In [24]:
# Vader Sentiment
analyzer = SentimentIntensityAnalyzer() 
sentiment = analyzer.polarity_scores(df).get('compound')
print('compound', sentiment)

compound 0.0


In [25]:
df['score'] = df.text.map(analyzer.polarity_scores).map(lambda x: x.get('compound'))
df.head(5)

,text,score
0,sundayfunday coronavirus style vino cheers https t co,0.4767
1,this pandemic has confirmed my worst fears most people dont know how to make entertaining videos sinceivebeenquarantined americasunfunniestvideos wrestlemania tonyaharding,-0.6124
2,is this true \nhttps t co \n ecuadorenemergencia coronaviruspandemic,0.4215
3,many us thought it was wuhan province but it could never be us then it was italy but it could never be us now it is here one newyorker died every minutes from over this weekend absolutely devastating \n\nhttps t co,-0.9230
4,ah coronavirus humor https t co,0.2732


[back to top](#top)

## 7 | Classification<a id='7'></a>  

### 7.1 Naive Bayes: Bernoulli<a id='71'></a> 

In [33]:
# BernoulliNB() 
bern = BernoulliNB().fit(X_train, y_train)
y_predict_bern = bern.predict(X_val) 


In [34]:
seven = ["Bernoulli NB", 'bern',
       recall_score(y_predict_bern, y_val),
       roc_auc_score(y_val, bern.predict_proba(X_val)[:,1])]

five

['Random forest', 'forest', 0.36295871559633025, 0.7973551116403413]

[back to top](#top)

### 7.2 Naive Bayes: Gaussian<a id='72'></a> 

In [35]:
gaus = GaussianNB().fit(X_train, y_train)
y_pred_gaus = gaus.predict(X_val)


In [36]:
eight = ["Gaussian NB",'gaus',
         recall_score(y_pred_gaus, y_val),
         roc_auc_score(y_val, gaus.predict_proba(X_val)[:,1])]
eight

['Gaussian NB', 'gaus', 0.26453726453726456, 0.8053429818657589]

[back to top](#top)

### 7.3 Naive Bayes: Multinomial<a id='73'></a> 

In [37]:
multi = MultinomialNB().fit(X_train, y_train)
y_pred_multi = multi.predict(X_val)


In [38]:
nine = ["Multinomial NB", 'multi',
        recall_score(y_pred_multi, y_val),
        roc_auc_score(y_val, multi.predict_proba(X_val)[:,1])]
nine

['Multinomial NB', 'multi', 0.21883163144146284, 0.7628424887440532]

#### [comment ]

[back to top](#top)

In [ ]:
# http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g1-nlp-overview/nlp_unsupervised_nlp_exercises5_soln.ipynb
# Our goal is to create a Naive Bayes model that will look at the review text and determine if the review is positive or negative. Let's start by prepping the data.

# # define the input and output of the model
# X = data.review
# y = data.sentiment

# # split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# X_train_cv = cv.fit_transform(X_train)
# X_test_cv  = cv.transform(X_test)

In [ ]:
# dtm_cv = pd.DataFrame(X_train_cv.toarray(), columns=cv.get_feature_names())
# dtm_cv

In [ ]:
# Next, we're going to put this document-term matrix through a Naive Bayes model and see how well the model performs.
# mnb = MultinomialNB()
# mnb.fit(X_train_cv, y_train)
# mnb.score(X_test_cv, y_test)

In [ ]:
# Using `CountVectorizer`, we are able to predict the sentiment of a review with 87.7% accuracy. Next, you are tasked with repeating the whole process again, but using `TfidfVectorizer` instead to see if you can get a better prediction score.

# 1. Create a `TfidfVectorizer` object with the same hyperparameters as the `CountVectorizer` object we created earlier and name it `tv`.

# 2. Take the `X_train` data and turn it into a TF-IDF matrix called `X_train_tv`.

# 3. Take the `X_test` data and turn it into a TF-IDF matrix with the same columns as `X_train_tv` and call it `X_test_tv`.

# 4. Turn `X_train_tv` into a pandas dataframe and call it `tfidf`.

# (See final step below)

In [ ]:
# tv = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
# X_train_tv = tv.fit_transform(X_train)
# X_test_tv  = tv.transform(X_test)
# tfidf = pd.DataFrame(X_train_tv.toarray(), columns=tv.get_feature_names())
# tfidf

In [ ]:
# 5. Fit a GaussianNB model and save the final score as `tfidf_score`.

# gnb = GaussianNB()
# gnb.fit(X_train_tv.toarray(), y_train)
# tfidf_score = gnb.score(X_test_tv.toarray(), y_test)
# tfidf_score

In [ ]:
# The final prediction accuracy using the TF-IDF Vectorizer was 84.5% versus the final prediction accuracy using the Count Vectorizer, which was 87.7%.

# This tells us that while TF-IDF can be the better option over simple word counts, it is not always the case. The best approach is to try both vectorizers and choose the one that works best for your dataset and analysis goal.

[back to top](#top)